Training of the model in the previous notebook was leveraging the GPU in case it was available. However, after training and closing of this notebook, information about the course of training and the development of the loss was gone. We would like to keep this information as it might be relevant for diagnostic purposes later on, such as convergence or overfitting.

A separate tool, originally developed within the tensorflow ecosystem but now adapted to pytorch, provides a solution for this: Tensorboard. More information is available [here](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html).

In [1]:
from torch.utils.tensorboard import SummaryWriter

In [2]:
import torch

In [3]:
cuda_present = torch.cuda.is_available()
ndevices = torch.cuda.device_count()
use_cuda = cuda_present and ndevices > 0
device = torch.device("cuda" if use_cuda else "cpu")  # "cuda:0" ... default device, "cuda:1" would be GPU index 1, "cuda:2" etc
print("number of devices:", ndevices, "\tchosen device:", device, "\tuse_cuda=", use_cuda)

number of devices: 0 	chosen device: cpu 	use_cuda= False


In [4]:
from torch.utils.data import DataLoader
from data import DSBData, get_dsb2018_train_files

In [5]:
from monai.networks.nets import BasicUNet

In [6]:
train_img_files, train_lbl_files = get_dsb2018_train_files()

train_data = DSBData(
    image_files=train_img_files,
    label_files=train_lbl_files,
    target_shape=(256, 256)
)

print(len(train_data))

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=1, pin_memory=True)

100%|█████████████████████████████████████████████████████████████████| 382/382 [00:14<00:00, 26.33it/s]

232


In [7]:
model = BasicUNet(
    spatial_dims=2,
    in_channels=1,
    out_channels=1,
    features=[16, 16, 32, 64, 128, 16],
    act="relu",
    norm="batch",
    dropout=0.25,
)

# transfer the model to the chosen device
model = model.to(device)

BasicUNet features: (16, 16, 32, 64, 128, 16).


Training of a neural network means updating its parameters (weights) using a strategy that involves the gradients of a loss function with respect to the model parameters in order to adjust model weights to minimize this loss.

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1.e-3)
init_params = list(model.parameters())[0].clone().detach()

Such a training is performed by iterating over the batches of the training dataset multiple times. Each full iteration over the dataset is termed an epoch.

During or after training the tensorboard logs can be visualized as follows: in a terminal, type

```shell
tensorboard --logdir "path/to/logs",
```

then open a browser on `localhost:6006` (or whichever port the tensorboard server outputted as running on).
Alternatively, tensorboard can be accessed from jupyter as well:

In [9]:
%load_ext tensorboard
%tensorboard --port 6006 --logdir ./logs

In [10]:
max_nepochs = 1
log_interval = 1
writer = SummaryWriter(log_dir="logs", comment="this is the test of SummaryWriter")

model.train(True)

# expects raw unnormalized scores and combines sigmoid + BCELoss for better
# numerical stability.
# expects B x C x W x D
loss_function = torch.nn.BCEWithLogitsLoss(reduction="mean")

for epoch in range(1, max_nepochs + 1):
    for batch_idx, (X, y) in enumerate(train_loader):
        # the inputs and labels have to be on the same device as the model
        X, y = X.to(device), y.to(device)
        
        optimizer.zero_grad()

        prediction_logits = model(X)
        
        batch_loss = loss_function(prediction_logits, y)

        batch_loss.backward()

        optimizer.step()

        if batch_idx % log_interval == 0:
            print(
                "Train Epoch:",
                epoch,
                "Batch:",
                batch_idx,
                "Total samples processed:",
                (batch_idx + 1) * train_loader.batch_size,
                "Loss:",
                batch_loss.item(),
            )
            writer.add_scalar("Loss/train", batch_loss.item(), batch_idx)
writer.close()

Train Epoch: 1 Batch: 0 Total samples processed: 32 Loss: 0.789273738861084
Train Epoch: 1 Batch: 1 Total samples processed: 64 Loss: 0.7346146106719971
Train Epoch: 1 Batch: 2 Total samples processed: 96 Loss: 0.6947914361953735
Train Epoch: 1 Batch: 3 Total samples processed: 128 Loss: 0.6753167510032654
Train Epoch: 1 Batch: 4 Total samples processed: 160 Loss: 0.6629210710525513
Train Epoch: 1 Batch: 5 Total samples processed: 192 Loss: 0.6484382152557373
Train Epoch: 1 Batch: 6 Total samples processed: 224 Loss: 0.6535567045211792
Train Epoch: 1 Batch: 7 Total samples processed: 256 Loss: 0.6217544078826904
